In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\User\\Desktop\\3-IDSD\\mlops\\MLOPS\\MlopsProject\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\User\\Desktop\\3-IDSD\\mlops\\MLOPS\\MlopsProject'

In [19]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    max_depth: int
    target_column: str

In [21]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [22]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForest
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            max_depth = params.max_depth,
            target_column = schema.name
            
        )

        return model_trainer_config

In [23]:
import pandas as pd
import os
from mlProject import logger
import joblib
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier

In [24]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        # 1️⃣ Charger les données
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # 2️⃣ Séparer X / y
        X_train = train_data.drop([self.config.target_column], axis=1)
        y_train = train_data[self.config.target_column]

        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[self.config.target_column]

        # 3️⃣ Appliquer SMOTE uniquement sur le train
        smote = SMOTE(random_state=42)
        X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

        logger.info("Target distribution AFTER SMOTE:")
        logger.info(pd.Series(y_train_res).value_counts().to_string())

        logger.info(f"Train before SMOTE: {X_train.shape}")
        logger.info(f"Train after SMOTE: {X_train_res.shape}")

        # 4️⃣ Entraîner RandomForest
        model = RandomForestClassifier(
            n_estimators=self.config.n_estimators,
            max_depth=self.config.max_depth,
            random_state=42,
            n_jobs=-1
        )

        model.fit(X_train_res, y_train_res)

        # 5️⃣ Sauvegarde du modèle
        os.makedirs(self.config.root_dir, exist_ok=True)
        model_path = os.path.join(self.config.root_dir, self.config.model_name)

        joblib.dump(model, model_path)

        logger.info(f"RandomForest model saved at {model_path}")


In [25]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-12-20 13:12:28,032: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-20 13:12:28,040: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-20 13:12:28,044: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-12-20 13:12:28,046: INFO: common: created directory at: artifacts]
[2025-12-20 13:12:28,047: INFO: common: created directory at: artifacts/model_trainer]
[2025-12-20 13:12:29,948: INFO: 2849920993: Target distribution AFTER SMOTE:]
[2025-12-20 13:12:29,981: INFO: 2849920993: RainTomorrow
0    88218
1    88218]
[2025-12-20 13:12:29,981: INFO: 2849920993: Train before SMOTE: (113754, 107)]
[2025-12-20 13:12:29,981: INFO: 2849920993: Train after SMOTE: (176436, 107)]
[2025-12-20 13:12:39,663: INFO: 2849920993: RandomForest model saved at artifacts/model_trainer\model.joblib]
